<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Capstone Project: Drowsiness Detection

# Content:

- Capstone Project Part 1 - Deep Learning
- Capstone Project Part 2 - Live Demo

# Capstone Project Part 2 - Live Demo

# Introduction:

One of the world's sleepiest nations is Singapore. In a recent poll, 43 cities were analyzed, and Singapore came out on top for having the least amount of sleep. Lack of sleep or low quality sleep are unacceptable. Whatever the source, a lack of sleep over an extended period of time will be detrimental to your overall wellbeing, productivity, and safety.

If you've ever driven, it's likely that you've dozed off at least once. Although we don't like to admit it, there is a serious problem that must be fixed because it has detrimental impacts. One in four car accidents are the result of drowsy driving, and one in twenty-five adult drivers admit to falling asleep at the wheel in the 30 days prior. The fact that tired driving involves more than just falling asleep behind the wheel is the most alarming element.

Given the significance of this problem, we believe it is essential to offer a sleepiness monitoring system, especially in the beginning to avoid accidents.

# Problem Statement:

Additionally, we believe that being drowsy can negatively impact people in both professional and academic situations. College and sleep deprivation go hand in hand, but occupational exhaustion, especially while operating heavy equipment, can result in catastrophic injuries, much like when driving while intoxicated.

We propose to solve this problem by developing a detection system that detects important symptoms of fatigue and raises an alarm when required. The main objective of this study is to create a sleepiness detection system that monitors the eyes; it is thought that by identifying the signs of driver fatigue early, an accident can be prevented. When this happens when drowsiness is found, a warning signal is sent to the driver to let them know. This detection device enables early identification of a reduction in driver alertness while driving and offers a noncontact technique for evaluating various levels of driver attentiveness. When this happens when weariness is found, a warning signal is sent to the driver to let them know.

In [1]:
#Required Packages

import dlib
import cv2
from imutils import face_utils
from scipy.spatial import distance 
import math
import pandas as pd
import numpy as np

from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score, f1_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics
import warnings
from sklearn import preprocessing
from statistics import mean

from scipy.spatial import distance as dist
from imutils.video import VideoStream

from threading import Thread
import playsound
import argparse
import time
import math


p = "68 face landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(p)

In [2]:
#Feature Functions

def eye_aspect_ratio(eye):
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

def mouth_aspect_ratio(mouth):
    A = distance.euclidean(mouth[14], mouth[18])
    C = distance.euclidean(mouth[12], mouth[16])
    mar = (A ) / (C)
    return mar

def circularity(eye):
    A = distance.euclidean(eye[1], eye[4])
    radius  = A/2.0
    Area = math.pi * (radius ** 2)
    p = 0
    p += distance.euclidean(eye[0], eye[1])
    p += distance.euclidean(eye[1], eye[2])
    p += distance.euclidean(eye[2], eye[3])
    p += distance.euclidean(eye[3], eye[4])
    p += distance.euclidean(eye[4], eye[5])
    p += distance.euclidean(eye[5], eye[0])
    return 4 * math.pi * Area /(p**2)

def mouth_over_eye(eye):
    ear = eye_aspect_ratio(eye)
    mar = mouth_aspect_ratio(eye)
    mouth_eye = mar/ear
    return mouth_eye

def lip_distance(shape):
    top_lip = shape[50:53]
    top_lip = np.concatenate((top_lip, shape[61:64]))

    low_lip = shape[56:59]
    low_lip = np.concatenate((low_lip, shape[65:68]))

    top_mean = np.mean(top_lip, axis=0)
    low_mean = np.mean(low_lip, axis=0)

    dist = abs(top_mean[1] - low_mean[1])
    return dist


def average(y_pred):
    for i in range(len(y_pred)):
        if i % 240 == 0 or (i+1) % 240 == 0:
            pass
        else: 
            average = float(y_pred[i-1] +  y_pred[i] + y_pred[i+1])/3
            if average >= 0.5:
                y_pred[i] = 1
            else:
                y_pred[i] = 0
    return y_pred

The first one only detect the eyes and when the eyes reach a threshold(manually set by ourselves), it will prompt that the user eyes is 'closing'

In [3]:
def justeyes():
    data = []
    cap = cv2.VideoCapture(0)

    while True:
        # Getting out image by webcam 
        _, image = cap.read()
        # Converting the image to gray scale
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        size = gray.shape

        # Get faces into webcam's image
        rects = detector(image, 0)

        # For each detected face, find the landmark.
        for (i, rect) in enumerate(rects):
            shape = predictor(gray, rect)
            shape = face_utils.shape_to_np(shape)#converting to NumPy Array
            # extract the left and right eye coordinates, then use the
            # coordinates to compute the eye aspect ratio for both eyes
            leftEye = shape[lStart:lEnd]
            rightEye = shape[rStart:rEnd]
            leftEAR = eye_aspect_ratio(leftEye)
            rightEAR = eye_aspect_ratio(rightEye)
            ear = (leftEAR + rightEAR) / 2.0

            # compute the convex hull for the left and right eye, then
            # visualize each of the eyes
            leftEyeHull = cv2.convexHull(leftEye)
            rightEyeHull = cv2.convexHull(rightEye)
            cv2.drawContours(image, [leftEyeHull], -1, (0, 255, 0), 1)
            cv2.drawContours(image, [rightEyeHull], -1, (0, 255, 0), 1)

            # check to see if the eye aspect ratio is below the blink
            # threshold, and if so, increment the blink frame counter
            if ear < EYE_THRESH:
                COUNTER += 1
                # if the eyes were closed for a sufficient number of times
                # then show the warning
                if COUNTER >= EYE_AR_CONSEC_FRAMES:
                    cv2.putText(image, "*************ALERT! EYES ClOSING! ****************", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                # otherwise, the eye aspect ratio is not below the blink
                # threshold, so reset the counter and alarm
            else:
                COUNTER = 0
        # Show the image
        cv2.imshow("Output", image)
            
        k = cv2.waitKey(5) & 0xFF
        if k == 27:
            break

    cv2.destroyAllWindows()
    cap.release()
    
    features_test = []
    for d in data:
        eye = d[36:68]
        ear = eye_aspect_ratio(eye)
        mar = mouth_aspect_ratio(eye)
        cir = circularity(eye)
        mouth_eye = mouth_over_eye(eye)
        features_test.append([ear, mar, cir, mouth_eye])
    
    features_test = np.array(features_test)
    x = features_test
    y = pd.DataFrame(x,columns=["EAR","MAR","Circularity","MOE"])
    df_means = y.mean(axis=0)
    df_std = y.std(axis=0)
    
    return df_means,df_std

font                   = cv2.FONT_HERSHEY_SIMPLEX
bottomLeftCornerOfText = (10,400)
fontScale              = 1
fontColor              = (255,255,255)
lineType               = 2

In [4]:
# Using only a threshold value for detecting drowsiness
EYE_THRESH = 0.22  # indicate for blink
EYE_FRAMES = 40  # minimum of consecutive frames of blinking
MOUTH_AR_THRESH = 1.2
YAWN_THRESH = 32
ALARM_ON = False
states = []
ear_results = []
# grab the indexes of the facial landmarks for the mouth
(mStart, mEnd) = (49, 68)

# grab the indexes of the facial landmarks for the left and
# right eye, respectively
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
EYE_AR_CONSEC_FRAMES = 3


#Run Calibration
#mean, std = justeyes()

Similar to the first one but this one we will include in the mouth and a 'yawn' popup will come out when the mouth is open (assuming that is a yawn)

In [5]:
def mouthandeyes2():
    data = []
    cap = cv2.VideoCapture(0)

    while True:
        # Getting out image by webcam 
        _, image = cap.read()
        # Converting the image to gray scale
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        size = gray.shape

        # Get faces into webcam's image
        rects = detector(image, 0)

        # For each detected face, find the landmark.
        for (i, rect) in enumerate(rects):
            # Make the prediction and transfom it to numpy array
            shape = predictor(gray, rect)
            shape = face_utils.shape_to_np(shape)
            
            # Draw on our image, all the finded cordinate points (x,y) 
            for (x, y) in shape:
                cv2.circle(image, (x, y), 2, (0, 255, 0), -1)
            # build features 
            eye = shape[36:68] # Extracting relevant parts (eyes to mouth)
            # calculate ear
            ear = eye_aspect_ratio(eye)
            # visualize contours only of the eyes
            leftEye  = shape[42:48]
            rightEye = shape[36:42]
            dist = lip_distance(shape)
            
            cv2.drawContours(image, [cv2.convexHull(leftEye)], -1, (0, 255, 0), 1)
            cv2.drawContours(image, [cv2.convexHull(rightEye)], -1, (0, 255, 0), 1)
            
            lip = shape[48:60]
            cv2.drawContours(image, [lip], -1, (0, 255, 0), 1)
        
            # check if ear is below defined threshold
            if ear < EYE_THRESH:
                    cv2.putText(image, "Drowsiness detected!", (10, 30), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                    
            
        
            # draw EAR
            cv2.putText(image, "EAR: {:3f}".format(ear), (300, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            
            mouth = shape[mStart:mEnd]
            mouthMAR = mouth_aspect_ratio(mouth)
            mar = mouthMAR
            # compute the convex hull for the mouth, then
            # visualize the mouth
            mouthHull = cv2.convexHull(mouth)

            cv2.drawContours(image, [mouthHull], -1, (0, 255, 0), 1)
            cv2.putText(image, "MAR: {:.2f}".format(mar), (650, 20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

            # Draw text if mouth is open
            #if mar > MOUTH_AR_THRESH:
                #cv2.putText(image, "Yawning!", (800, 20),
                            #cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            
            if dist > YAWN_THRESH:
                cv2.putText(image, "Yawn Alert", (800, 20),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        
        # Show the image
        cv2.imshow("Output", image)
            

        k = cv2.waitKey(5) & 0xFF
        if k == 27:
            break

    cv2.destroyAllWindows()
    cap.release()
    
    features_test = []
    for d in data:
        eye = d[36:68]
        ear = eye_aspect_ratio(eye)
        mar = mouth_aspect_ratio(eye)
        cir = circularity(eye)
        mouth_eye = mouth_over_eye(eye)
        features_test.append([ear, mar, cir, mouth_eye])
    
    features_test = np.array(features_test)
    x = features_test
    y = pd.DataFrame(x,columns=["EAR","MAR","Circularity","MOE"])
    df_means = y.mean(axis=0)
    df_std = y.std(axis=0)
    
    return df_means,df_std

font                   = cv2.FONT_HERSHEY_SIMPLEX
bottomLeftCornerOfText = (10,400)
fontScale              = 1
fontColor              = (255,255,255)
lineType               = 2

In [6]:
# Using only a threshold value for detecting drowsiness
EYE_THRESH = 0.22  # indicate for blink
EYE_FRAMES = 40  # minimum of consecutive frames of blinking
MOUTH_AR_THRESH = 1.2
YAWN_THRESH = 32
ALARM_ON = False
states = []
ear_results = []
# grab the indexes of the facial landmarks for the mouth
(mStart, mEnd) = (49, 68)

#Run Calibration
#mean, std = mouthandeyes2()

In [7]:
# 3D model points.
model_points = np.array([
    (0.0, 0.0, 0.0),             # Nose tip 34
    (0.0, -330.0, -65.0),        # Chin 9
    (-225.0, 170.0, -135.0),     # Left eye left corner 37
    (225.0, 170.0, -135.0),      # Right eye right corne 46
    (-150.0, -150.0, -125.0),    # Left Mouth corner 49
    (150.0, -150.0, -125.0)      # Right mouth corner 55
])


# Checks if a matrix is a valid rotation matrix.
def isRotationMatrix(R):
    Rt = np.transpose(R)
    shouldBeIdentity = np.dot(Rt, R)
    I = np.identity(3, dtype=R.dtype)
    n = np.linalg.norm(I - shouldBeIdentity)
    return n < 1e-6


# Calculates rotation matrix to euler angles
# The result is the same as MATLAB except the order
# of the euler angles ( x and z are swapped ).
def rotationMatrixToEulerAngles(R):
    assert(isRotationMatrix(R))
    sy = math.sqrt(R[0, 0] * R[0, 0] + R[1, 0] * R[1, 0])
    singular = sy < 1e-6
    if not singular:
        x = math.atan2(R[2, 1], R[2, 2])
        y = math.atan2(-R[2, 0], sy)
        z = math.atan2(R[1, 0], R[0, 0])
    else:
        x = math.atan2(-R[1, 2], R[1, 1])
        y = math.atan2(-R[2, 0], sy)
        z = 0
    return np.array([x, y, z])


def getHeadTiltAndCoords(size, image_points, frame_height):
    focal_length = size[1]
    center = (size[1]/2, size[0]/2)
    camera_matrix = np.array([[focal_length, 0, center[0]], [
        0, focal_length, center[1]], [0, 0, 1]], dtype="double")
    # print "Camera Matrix :\n {0}".format(camera_matrix)

    dist_coeffs = np.zeros((4, 1))  # Assuming no lens distortion
    (_, rotation_vector, translation_vector) = cv2.solvePnP(model_points, image_points,
                                                                  camera_matrix, dist_coeffs, 
                                                                  flags = cv2.SOLVEPNP_ITERATIVE)  # flags=cv2.CV_ITERATIVE)

    # print "Rotation Vector:\n {0}".format(rotation_vector)
    # print "Translation Vector:\n {0}".format(translation_vector)
    # Project a 3D point (0, 0 , 1000.0) onto the image plane
    # We use this to draw a line sticking out of the nose_end_point2D
    (nose_end_point2D, _) = cv2.projectPoints(np.array(
        [(0.0, 0.0, 1000.0)]), rotation_vector, translation_vector, camera_matrix, dist_coeffs)

    #get rotation matrix from the rotation vector
    rotation_matrix, _ = cv2.Rodrigues(rotation_vector)

    #calculate head tilt angle in degrees
    head_tilt_degree = abs(
        [-180] - np.rad2deg([rotationMatrixToEulerAngles(rotation_matrix)[0]]))

    #calculate starting and ending points for the two lines for illustration
    starting_point = (int(image_points[0][0]), int(image_points[0][1]))
    ending_point = (int(nose_end_point2D[0][0][0]), int(nose_end_point2D[0][0][1]))

    ending_point_alternate = (ending_point[0], frame_height // 2)

    return head_tilt_degree, starting_point, ending_point, ending_point_alternate

In [8]:
def mouthandeyes3():
    data = []
    cap = cv2.VideoCapture(0)

    while True:
        # Getting out image by webcam 
        _, frame = cap.read()
        # Converting the image to gray scale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        size = gray.shape

        # Get faces into webcam's image
        rects = detector(frame, 0)
        
        
        if len(rects) > 0:
            text = "{} face(s) found".format(len(rects))
            cv2.putText(frame, text, (10, 20),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        
    
        # loop over the face detections
        for rect in rects:
            # compute the bounding box of the face and draw it on the
            # frame
            (bX, bY, bW, bH) = face_utils.rect_to_bb(rect)
            cv2.rectangle(frame, (bX, bY), (bX + bW, bY + bH), (0, 255, 0), 1)
            # determine the facial landmarks for the face region, then
            # convert the facial landmark (x, y)-coordinates to a NumPy
            # array
            shape = predictor(gray, rect)
            shape = face_utils.shape_to_np(shape)

            # extract the left and right eye coordinates, then use the
            # coordinates to compute the eye aspect ratio for both eyes
            leftEye = shape[lStart:lEnd]
            rightEye = shape[rStart:rEnd]
            leftEAR = eye_aspect_ratio(leftEye)
            rightEAR = eye_aspect_ratio(rightEye)
            # average the eye aspect ratio together for both eyes
            ear = (leftEAR + rightEAR) / 2.0
            
            dist = lip_distance(shape)

            # compute the convex hull for the left and right eye, then
            # visualize each of the eyes
            leftEyeHull = cv2.convexHull(leftEye)
            rightEyeHull = cv2.convexHull(rightEye)
            cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
            cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
        
            # check to see if the eye aspect ratio is below the blink
            # threshold, and if so, increment the blink frame counter
            if ear < EYE_AR_THRESH:
                COUNTER += 1
                # if the eyes were closed for a sufficient number of times
                # then show the warning
                if COUNTER >= EYE_AR_CONSEC_FRAMES:
                    cv2.putText(frame, "Eyes Closing!", (500, 20),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                # otherwise, the eye aspect ratio is not below the blink
                # threshold, so reset the counter and alarm
            else:
                COUNTER = 0

            mouth = shape[mStart:mEnd]

            mouthMAR = mouth_aspect_ratio(mouth)
            mar = mouthMAR
            # compute the convex hull for the mouth, then
            # visualize the mouth
            mouthHull = cv2.convexHull(mouth)

            cv2.drawContours(frame, [mouthHull], -1, (0, 255, 0), 1)
            cv2.putText(frame, "MAR: {:.2f}".format(mar), (650, 20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        
            # Draw text if mouth is open
            #if mar > MOUTH_AR_THRESH:
                #cv2.putText(frame, "Yawning!", (800, 20),
                            #cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            if dist > YAWN_THRESH:
                cv2.putText(frame, "Yawn Alert", (800, 20),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    
            # loop over the (x, y)-coordinates for the facial landmarks
            # and draw each of them
            for (i, (x, y)) in enumerate(shape):
                if i == 33:
                    # something to our key landmarks
                    # save to our new key point list
                    # i.e. keypoints = [(i,(x,y))]
                    image_points[0] = np.array([x, y], dtype='double')
                    # write on frame in Green
                    cv2.circle(frame, (x, y), 1, (0, 255, 0), -1)
                    cv2.putText(frame, str(i + 1), (x - 10, y - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 255, 0), 1)
                elif i == 8:
                    # something to our key landmarks
                    # save to our new key point list
                    # i.e. keypoints = [(i,(x,y))]
                    image_points[1] = np.array([x, y], dtype='double')
                    # write on frame in Green
                    cv2.circle(frame, (x, y), 1, (0, 255, 0), -1)
                    cv2.putText(frame, str(i + 1), (x - 10, y - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 255, 0), 1)
                
                elif i == 36:
                    # something to our key landmarks
                    # save to our new key point list
                    # i.e. keypoints = [(i,(x,y))]
                    image_points[2] = np.array([x, y], dtype='double')
                    # write on frame in Green
                    cv2.circle(frame, (x, y), 1, (0, 255, 0), -1)
                    cv2.putText(frame, str(i + 1), (x - 10, y - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 255, 0), 1)
                elif i == 45:
                    # something to our key landmarks
                    # save to our new key point list
                    # i.e. keypoints = [(i,(x,y))]
                    image_points[3] = np.array([x, y], dtype='double')
                    # write on frame in Green
                    cv2.circle(frame, (x, y), 1, (0, 255, 0), -1)
                    cv2.putText(frame, str(i + 1), (x - 10, y - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 255, 0), 1)
                elif i == 48:
                    # something to our key landmarks
                    # save to our new key point list
                    # i.e. keypoints = [(i,(x,y))]
                    image_points[4] = np.array([x, y], dtype='double')
                    # write on frame in Green
                    cv2.circle(frame, (x, y), 1, (0, 255, 0), -1)
                    cv2.putText(frame, str(i + 1), (x - 10, y - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 255, 0), 1)
                
                elif i == 54:
                    # something to our key landmarks
                    # save to our new key point list
                    # i.e. keypoints = [(i,(x,y))]
                    image_points[5] = np.array([x, y], dtype='double')
                    # write on frame in Green
                    cv2.circle(frame, (x, y), 1, (0, 255, 0), -1)
                    cv2.putText(frame, str(i + 1), (x - 10, y - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 255, 0), 1)
                else:
                    # everything to all other landmarks
                    # write on frame in Red
                    cv2.circle(frame, (x, y), 1, (0, 0, 255), -1)
                    cv2.putText(frame, str(i + 1), (x - 10, y - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 0, 255), 1)

            #Draw the determinant image points onto the person's face
            for p in image_points:
                cv2.circle(frame, (int(p[0]), int(p[1])), 3, (0, 0, 255), -1)

            (head_tilt_degree, start_point, end_point, 
                end_point_alt) = getHeadTiltAndCoords(size, image_points, frame_height)

            cv2.line(frame, start_point, end_point, (255, 0, 0), 2)
            cv2.line(frame, start_point, end_point_alt, (0, 0, 255), 2)
        
            if head_tilt_degree:
                cv2.putText(frame, 'Head Tilt Degree: ' + str(head_tilt_degree[0]), (170, 20),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

            # extract the mouth coordinates, then use the
            # coordinates to compute the mouth aspect ratio
        # show the frameq
        cv2.imshow("Frame", frame)
        key = cv2.waitKey(1) & 0xFF

        # if the `q` key was pressed, break from the loop
        if key == ord("q"):
            break

    cv2.destroyAllWindows()
    cap.release()
    
    features_test = []
    for d in data:
        eye = d[36:68]
        ear = eye_aspect_ratio(eye)
        mar = mouth_aspect_ratio(eye)
        cir = circularity(eye)
        mouth_eye = mouth_over_eye(eye)
        features_test.append([ear, mar, cir, mouth_eye])
    
    features_test = np.array(features_test)
    x = features_test
    y = pd.DataFrame(x,columns=["EAR","MAR","Circularity","MOE"])
    df_means = y.mean(axis=0)
    df_std = y.std(axis=0)
    
    return df_means,df_std

font                   = cv2.FONT_HERSHEY_SIMPLEX
bottomLeftCornerOfText = (10,400)
fontScale              = 1
fontColor              = (255,255,255)
lineType               = 2

In [9]:
# 400x225 to 1024x576
frame_width = 1024
frame_height = 576

# Using only a threshold value for detecting drowsiness
EYE_THRESH = 0.22  # indicate for blink
EYE_FRAMES = 40  # minimum of consecutive frames of blinking
MOUTH_AR_THRESH = 1.2
YAWN_THRESH = 32
ALARM_ON = False
states = []
ear_results = []

EYE_AR_THRESH = 0.22
MOUTH_AR_THRESH = 1.2
EYE_AR_CONSEC_FRAMES = 3



def main():
    global COUNTER

yy = main()



# loop over the frames from the video stream
# 2D image points. If you change the image, you need to change vector
image_points = np.array([
    (359, 391),     # Nose tip 34
    (399, 561),     # Chin 9
    (337, 297),     # Left eye left corner 37
    (513, 301),     # Right eye right corne 46
    (345, 465),     # Left Mouth corner 49
    (453, 469)      # Right mouth corner 55
], dtype="double")

(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

# grab the indexes of the facial landmarks for the mouth
(mStart, mEnd) = (49, 68)

#Run Calibration
#mean, std = mouthandeyes3()
# initialize the video stream and sleep for a bit, allowing the
# camera sensor to warm up
print("[INFO] initializing camera...")

[INFO] initializing camera...


In [10]:
def mouthandeyes4():
    data = []
    cap = cv2.VideoCapture(0)

    while True:
        # Getting out image by webcam 
        _, frame = cap.read()
        # Converting the image to gray scale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        size = gray.shape

        # Get faces into webcam's image
        rects = detector(frame, 0)
        
        
        if len(rects) > 0:
            text = "{} face(s) found".format(len(rects))
            cv2.putText(frame, text, (10, 20),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        
    
        # loop over the face detections
        for rect in rects:
            # compute the bounding box of the face and draw it on the
            # frame
            (bX, bY, bW, bH) = face_utils.rect_to_bb(rect)
            cv2.rectangle(frame, (bX, bY), (bX + bW, bY + bH), (0, 255, 0), 1)
            # determine the facial landmarks for the face region, then
            # convert the facial landmark (x, y)-coordinates to a NumPy
            # array
            shape = predictor(gray, rect)
            shape = face_utils.shape_to_np(shape)

            # extract the left and right eye coordinates, then use the
            # coordinates to compute the eye aspect ratio for both eyes
            leftEye = shape[lStart:lEnd]
            rightEye = shape[rStart:rEnd]
            leftEAR = eye_aspect_ratio(leftEye)
            rightEAR = eye_aspect_ratio(rightEye)
            # average the eye aspect ratio together for both eyes
            ear = (leftEAR + rightEAR) / 2.0
            
            dist = lip_distance(shape)

            # compute the convex hull for the left and right eye, then
            # visualize each of the eyes
            leftEyeHull = cv2.convexHull(leftEye)
            rightEyeHull = cv2.convexHull(rightEye)
            cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
            cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
        
            # check to see if the eye aspect ratio is below the blink
            # threshold, and if so, increment the blink frame counter
            if ear < HIGH_EYE_AR_THRESH:
                COUNTER += 1
                # if the eyes were closed for a sufficient number of times
                # then show the warning
                if COUNTER >= HIGH_EYE_AR_CONSEC_FRAMES:
                    cv2.putText(frame, "High Alert!! Eyes Closing!", (500, 40),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                # otherwise, the eye aspect ratio is not below the blink
                    
            elif ear < LOW_EYE_AR_THRESH:
                COUNTER += 1
                # if the eyes were closed for a sufficient number of times
                # then show the warning
                if COUNTER >= LOW_EYE_AR_CONSEC_FRAMES:
                    cv2.putText(frame, "Alert! Eyes Closing!", (500, 40),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                # otherwise, the eye aspect ratio is not below the blink
                # threshold, so reset the counter and alarm    
            else:
                COUNTER = 0
            

            mouth = shape[mStart:mEnd]

            mouthMAR = mouth_aspect_ratio(mouth)
            mar = mouthMAR
            # compute the convex hull for the mouth, then
            # visualize the mouth
            mouthHull = cv2.convexHull(mouth)

            cv2.drawContours(frame, [mouthHull], -1, (0, 255, 0), 1)
            cv2.putText(frame, "MAR: {:.2f}".format(mar), (650, 20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        
            # Draw text if mouth is open
            #if mar > MOUTH_AR_THRESH:
                #cv2.putText(frame, "Yawning!", (800, 20),
                            #cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            if dist > HIGH_YAWN_THRESH:
                COUNT_YAWN += 1
                # if the mouth was openedd for a sufficient number of times
                # then show the warning
                if COUNT_YAWN >= HIGH_COUNT_YAWN:
                    cv2.putText(frame, "High Alert!! Excessive Yawn Alert", (800, 20),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            
            elif dist > LOW_YAWN_THRESH:
                cv2.putText(frame, "Alert!! Yawn Alert", (800, 20),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            
            else:
                COUNT_YAWN = 0
            
            # loop over the (x, y)-coordinates for the facial landmarks
            # and draw each of them
            for (i, (x, y)) in enumerate(shape):
                if i == 33:
                    # something to our key landmarks
                    # save to our new key point list
                    # i.e. keypoints = [(i,(x,y))]
                    image_points[0] = np.array([x, y], dtype='double')
                    # write on frame in Green
                    cv2.circle(frame, (x, y), 1, (0, 255, 0), -1)
                    cv2.putText(frame, str(i + 1), (x - 10, y - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 255, 0), 1)
                elif i == 8:
                    # something to our key landmarks
                    # save to our new key point list
                    # i.e. keypoints = [(i,(x,y))]
                    image_points[1] = np.array([x, y], dtype='double')
                    # write on frame in Green
                    cv2.circle(frame, (x, y), 1, (0, 255, 0), -1)
                    cv2.putText(frame, str(i + 1), (x - 10, y - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 255, 0), 1)
                
                elif i == 36:
                    # something to our key landmarks
                    # save to our new key point list
                    # i.e. keypoints = [(i,(x,y))]
                    image_points[2] = np.array([x, y], dtype='double')
                    # write on frame in Green
                    cv2.circle(frame, (x, y), 1, (0, 255, 0), -1)
                    cv2.putText(frame, str(i + 1), (x - 10, y - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 255, 0), 1)
                elif i == 45:
                    # something to our key landmarks
                    # save to our new key point list
                    # i.e. keypoints = [(i,(x,y))]
                    image_points[3] = np.array([x, y], dtype='double')
                    # write on frame in Green
                    cv2.circle(frame, (x, y), 1, (0, 255, 0), -1)
                    cv2.putText(frame, str(i + 1), (x - 10, y - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 255, 0), 1)
                elif i == 48:
                    # something to our key landmarks
                    # save to our new key point list
                    # i.e. keypoints = [(i,(x,y))]
                    image_points[4] = np.array([x, y], dtype='double')
                    # write on frame in Green
                    cv2.circle(frame, (x, y), 1, (0, 255, 0), -1)
                    cv2.putText(frame, str(i + 1), (x - 10, y - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 255, 0), 1)
                
                elif i == 54:
                    # something to our key landmarks
                    # save to our new key point list
                    # i.e. keypoints = [(i,(x,y))]
                    image_points[5] = np.array([x, y], dtype='double')
                    # write on frame in Green
                    cv2.circle(frame, (x, y), 1, (0, 255, 0), -1)
                    cv2.putText(frame, str(i + 1), (x - 10, y - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 255, 0), 1)
                else:
                    # everything to all other landmarks
                    # write on frame in Red
                    cv2.circle(frame, (x, y), 1, (0, 0, 255), -1)
                    cv2.putText(frame, str(i + 1), (x - 10, y - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 0, 255), 1)

            #Draw the determinant image points onto the person's face
            for p in image_points:
                cv2.circle(frame, (int(p[0]), int(p[1])), 3, (0, 0, 255), -1)

            (head_tilt_degree, start_point, end_point, 
                end_point_alt) = getHeadTiltAndCoords(size, image_points, frame_height)

            cv2.line(frame, start_point, end_point, (255, 0, 0), 2)
            cv2.line(frame, start_point, end_point_alt, (0, 0, 255), 2)
        
            if head_tilt_degree:
                cv2.putText(frame, 'Head Tilt Degree: ' + str(head_tilt_degree[0]), (170, 20),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

            # extract the mouth coordinates, then use the
            # coordinates to compute the mouth aspect ratio
        # show the frameq
        cv2.imshow("Frame", frame)
        key = cv2.waitKey(1) & 0xFF

        # if the `q` key was pressed, break from the loop
        if key == ord("q"):
            break

    cv2.destroyAllWindows()
    cap.release()
    
    features_test = []
    for d in data:
        eye = d[36:68]
        ear = eye_aspect_ratio(eye)
        mar = mouth_aspect_ratio(eye)
        cir = circularity(eye)
        mouth_eye = mouth_over_eye(eye)
        features_test.append([ear, mar, cir, mouth_eye])
    
    features_test = np.array(features_test)
    x = features_test
    y = pd.DataFrame(x,columns=["EAR","MAR","Circularity","MOE"])
    df_means = y.mean(axis=0)
    df_std = y.std(axis=0)
    
    return df_means,df_std

font                   = cv2.FONT_HERSHEY_SIMPLEX
bottomLeftCornerOfText = (10,400)
fontScale              = 1
fontColor              = (255,255,255)
lineType               = 2

In [ ]:
# 400x225 to 1024x576
frame_width = 1024
frame_height = 576

# (Low drowsiness Mode) Using only a threshold value for detecting drowsiness
LOW_EYE_THRESH = 0.34  # indicate for blink
LOW_EYE_FRAMES = 40  # minimum of consecutive frames of blinking
LOW_MOUTH_AR_THRESH = 1.2
LOW_YAWN_THRESH = 35
ALARM_ON = False
states = []
ear_results = []

LOW_EYE_AR_THRESH = 0.22
LOW_MOUTH_AR_THRESH = 1.2
LOW_EYE_AR_CONSEC_FRAMES = 6
LOW_COUNT_YAWN = 0


# (High drowsiness Mode) Using only a threshold value for detecting drowsiness
HIGH_EYE_THRESH = 0.22  # indicate for blink
HIGH_EYE_FRAMES = 40  # minimum of consecutive frames of blinking
HIGH_MOUTH_AR_THRESH = 1.2
HIGH_YAWN_THRESH = 50
ALARM_ON = False
states = []
ear_results = []

HIGH_EYE_AR_THRESH = 0.22
HIGH_MOUTH_AR_THRESH = 1.2
HIGH_EYE_AR_CONSEC_FRAMES = 3
HIGH_COUNT_YAWN = 5 


COUNT_YAWN = 0

def main():
    global COUNTER

yy = main()



# loop over the frames from the video stream
# 2D image points. If you change the image, you need to change vector
image_points = np.array([
    (359, 391),     # Nose tip 34
    (399, 561),     # Chin 9
    (337, 297),     # Left eye left corner 37
    (513, 301),     # Right eye right corne 46
    (345, 465),     # Left Mouth corner 49
    (453, 469)      # Right mouth corner 55
], dtype="double")

(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

# grab the indexes of the facial landmarks for the mouth
(mStart, mEnd) = (49, 68)

#Run Calibration
mean, std = mouthandeyes4()
# initialize the video stream and sleep for a bit, allowing the
# camera sensor to warm up
print("[INFO] initializing camera...")